In [1]:
import pandas as pd

# Food & Nutrition

In [2]:
hh = pd.read_parquet('data/hh_details.parquet')
hh.head()

,hh_size,employed_annual,nco_2015,nic_2008,max_income_from,hasincome_selfemp_agri,hasincome_wage_agri,hasincome_casual_agri,hh_type,hoh_religion,...,upper primary/middle,used_internet,days_outside_home,meals_daily,meals_school,meals_employer,meals_others,meals_payment,meals_home,mult
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,4,yes,112.0,47522.0,self-employment,NaN,NaN,NaN,1,Hinduism,...,0.0,4,0.0,12.0,0.0,0.0,3.0,10.0,347.0,57436
22300201,4,yes,112.0,73100.0,self-employment,NaN,NaN,NaN,1,Hinduism,...,2.0,3,0.0,12.0,0.0,0.0,0.0,10.0,350.0,27497
22300202,2,no,NaN,NaN,None,NaN,NaN,NaN,9,Hinduism,...,0.0,2,0.0,6.0,0.0,0.0,0.0,0.0,180.0,27497
22300203,2,no,NaN,NaN,None,NaN,NaN,NaN,9,Hinduism,...,1.0,0,0.0,6.0,0.0,0.0,6.0,0.0,174.0,27497
22300204,2,yes,231.0,85301.0,regular wage/salary earning,NaN,NaN,NaN,2,Christianity,...,0.0,2,0.0,6.0,0.0,0.0,3.0,2.0,175.0,27497


In [3]:
cols = ['rationcard_type', 'has_benefited_from_pmgky', 'meals_school', 'meals_others']
food = hh[cols]

In [4]:
proc = pd.read_parquet('data/hh-procurement.parquet')
proc.drop([c for c in proc if c.startswith('online_')], axis=1, inplace=True)
proc.head()

,used_ration_card,procured_rice,procured_wheat,procured_grain,procured_sugar,procured_pulses,procured_edible_oil,procured_other,performed_ceremony,meals_served_to_nonhh,...,has_mobile,has_bicycle,has_bike,has_car,has_truck,has_animalcart,has_fridge,has_washingmachine,has_ac,type_tv
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,no,False,False,False,False,False,False,False,no,5,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0
22300201,no,False,False,False,False,False,False,False,no,3,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN
22300202,no,False,False,False,False,False,False,False,no,2,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0
22300203,no,False,False,False,False,False,False,False,no,3,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0
22300204,no,False,False,False,False,False,False,False,no,3,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0


In [5]:
ration_cols = ['used_ration_card', 'procured_rice', 'procured_wheat', 'procured_pulses',
               'procured_grain', 'procured_sugar', 'procured_edible_oil', 'procured_other',
               # 'procured_kerosene_ration'
              ]
food = pd.concat([food, proc[ration_cols]], axis=1, verify_integrity=True)

In [7]:
sources = pd.read_parquet('data/baskets-sources.parquet')
qty = pd.read_parquet('data/baskets-consumption-value.parquet')

In [21]:
# cereals, pulses, sugar, salt, oil, meals
item_codes = [
    61,  # rice - free
    62,  # wheat / atta - free
    70,  # coarse grains, free
    101, # rice - pds purchase or free
    107, # wheat / atta pds purchase or free
    1,   # coarse grains - pds,
    71,  # pulses free
    158, # pulses pds
    72,  # gram free
    73,  # salt free
    74,  # sugar free
    178, # salt - pds
    171, # sugar - pds
    188, # edible oil pds
    75,  # edible oil, free
    76,  # subsidized cooked meals
]

In [23]:
food = pd.concat([food, sources[item_codes]], axis=1, verify_integrity=True)

In [25]:
food.to_parquet('data/food-indicators.parquet')

/home/jaidevd/conda/lib/python3.9/site-packages/pandas/io/parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


# Utilities

In [31]:
utils = pd.concat([
    hh[
        ['energy_source_cooking', 'energy_source_lighting']
    ],
    proc[
        ['used_ration_card', 'procured_kerosene_ration', 'received_subsidy_lpg', 'received_free_electricity']
    ],
    sources[334],  # kerosense - PDS
], axis=1, verify_integrity=True)
utils.head()

,energy_source_cooking,energy_source_lighting,used_ration_card,procured_kerosene_ration,received_subsidy_lpg,received_free_electricity,334
hhid,,,,,,,
22300101,LPG,electricity (incl. generated by solar or wind ...,no,2,1,2,NaN
22300201,LPG,electricity (incl. generated by solar or wind ...,no,2,1,2,NaN
22300202,LPG,electricity (incl. generated by solar or wind ...,no,2,1,2,NaN
22300203,LPG,electricity (incl. generated by solar or wind ...,no,2,2,2,NaN
22300204,LPG,electricity (incl. generated by solar or wind ...,no,2,1,2,NaN


In [32]:
utils.to_parquet('data/utilities-indicators.parquet')

/home/jaidevd/conda/lib/python3.9/site-packages/pandas/io/parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


# Health

In [37]:
proc[['is_hhmem_pmjay', 'is_hospitalization', 'is_benefit_healthscheme']].to_parquet('data/health-indicators.parquet')

# Education

In [41]:
proc[['is_hhmem_edu', 'received_free_textbook', 'received_free_stationary', 'received_free_schoolbag', 'received_free_oth_school', 'received_fee_waiver']].to_parquet('data/edu-indicators.parquet')

# Assets

In [46]:
proc[['received_free_laptop',
 'received_free_tab',
 'received_free_mobile',
 'received_free_bicycle',
 'received_free_bike',
 'received_free_clothing',
 'received_free_footwear',
 'received_free_oth_dgq']].to_parquet('data/assets-indicators.parquet')